In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


In [3]:
for name, _ in model.named_children():
    print(name)

conv1
pool
conv2
fc1
fc2
fc3


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [8]:
outputs= []
def pool_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(pool_hook)

In [9]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [10]:
outputs

[tensor([[[[0.7244, 0.2039, 1.0381,  ..., 1.1438, 0.2033, 0.7959],
           [0.4592, 0.4618, 0.6628,  ..., 1.0921, 1.0892, 0.6397],
           [0.8009, 0.5159, 0.8050,  ..., 0.5757, 0.9869, 0.7935],
           ...,
           [0.4200, 0.6199, 0.8591,  ..., 0.8165, 0.5911, 0.7667],
           [0.7698, 0.3865, 1.1263,  ..., 1.2081, 0.5062, 0.6811],
           [0.7467, 0.9559, 0.2879,  ..., 0.4444, 1.3940, 0.3484]],
 
          [[1.9603, 0.8175, 0.4541,  ..., 0.1001, 1.0970, 1.2584],
           [0.2441, 0.4517, 1.4506,  ..., 0.3435, 0.1645, 0.3157],
           [0.5703, 0.5193, 0.4272,  ..., 0.6088, 0.6968, 0.3504],
           ...,
           [0.7063, 0.3277, 0.3909,  ..., 0.0000, 0.6504, 1.2399],
           [0.8946, 0.1011, 0.9913,  ..., 0.0000, 0.9044, 0.2986],
           [1.9385, 0.6313, 0.3360,  ..., 0.9432, 1.5704, 0.5764]],
 
          [[0.8686, 1.4885, 0.8324,  ..., 1.9520, 0.8830, 0.4308],
           [1.0418, 0.5247, 0.4483,  ..., 0.6628, 0.6694, 1.0149],
           [1.1815, 0.95

In [11]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [14]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.ones_(m.bias)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [15]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


